In [72]:
# Импорт зависимостей
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD

In [74]:
#Путь к папкам с данными для обучения
"""
В папках (train, val) должны находиться папки, имя которых - название класса
пример:
/train
    - /cats
    - /dogs
/val
    - /cats
    - /dogs
Количество данных в обучающей выборке может быть неравномерным
В валидационной выборке как правило делают равномерное распределение (к примеру,по 50 снимков на класс)
"""
train_dir = "/train"
val_dir = "/val"

# Размер изображения для вохдного тензора нейросети (см. документацию по НС https://www.tensorflow.org/api_docs/python/tf/keras/applications)
"""Если при обучении используются предобученные веса (imagenet...), то размер входного тензора должен соответсвовать весам"""
img_width, img_height = 224, 224

# Входной тензор
input_shape = (img_width, img_height,3)

# Количество классов для обучения
num_classes = 3

# Количество эпох обучения
epochs = 50

# Количество изображений, которые будут подаваться НС при одной итерации обучения
batch_size = 18

In [75]:
# В качестве основной модели загружаем модель MobileNet с предобученными весами imagenet
base_model = tf.keras.applications.MobileNet(
    input_shape=(img_width, img_height, 3),
    include_top=False, #True
    weights='imagenet',
    classes=num_classes,
)
# Включаем переобучение основной модели
base_model.trainable = True
# Добавляем слои полносвязного слоя в модель
model = Sequential()
model.add(base_model)
model.add(Dropout(0.1))
model.add(GlobalAveragePooling2D(name='global_average_pooling2d'))
model.add(Dropout(0.1))
model.add(Dense(num_classes, activation='softmax', name='predictions'))

# Пример с другим вариантом полносвязного слоя
# model.add(base_model)
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(tf.keras.layers.Dense(num_classes, activation='softmax', name='predictions'))

# model.layers[0].trainable = False
# Ещё раз включаем переобучение для всех слоёв НС
for cnn_block_layer in model.layers[0].layers:
   cnn_block_layer.trainable = True
model.layers[0].trainable = True

In [76]:
# Скорость обучения НС 
"""
Если несколько эпох подряд на старте обучения получаются одинаковые значения точности примерно равные доле от 100/количество классов
Например: при 3 классах точность держится в районе 0.33
https://machinelearningmastery.com/using-learning-rate-schedules-deep-learning-models-python-keras/
"""
learning_rate = 0.00001
# Параметры специфичные для оптимизатора SGD 
decay_rate = learning_rate / epochs
momentum = 0.8
# Другие оптимизаторы https://keras.io/api/optimizers/
sgd = SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
# Компиляция модели
"""
Функции потерь: https://keras.io/api/losses/
Метрики точности: https://keras.io/api/metrics/
"""
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, 
              metrics=['categorical_accuracy'])

In [77]:
# Вывод структуры полученной модели
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Function (None, 7, 7, 1024)        3228864   
_________________________________________________________________
dropout_6 (Dropout)          (None, 7, 7, 1024)        0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 1024)              0         
_________________________________________________________________
predictions (Dense)          (None, 3)                 3075      
Total params: 3,231,939
Trainable params: 3,210,051
Non-trainable params: 21,888
_________________________________________________________________


In [78]:
# Генератор данных 
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
datagen = ImageDataGenerator(rescale=1. / 255, 
    validation_split=0,
    vertical_flip=True,
    horizontal_flip=False,
    fill_mode='nearest')

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size = (img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=1
)

validation_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    seed=1)


Found 3311 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [79]:
# Директория, в которую будут сохраняться лучшие эпохи обучения
cpt_dir = "chkpt/N_H_L_MobileNet"  
# Балансировка весов в обучающей выборке при помощи пакета sklearn
"""
Балансировка весов помогает избежать "перевеса" НС в сторону одного из классов при неравномерном распределении количества данных в разных классах
"""
from sklearn.utils import class_weight
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_generator.classes),
                                                 train_generator.classes)
# Приведение к необходимому формату
class_weights = {i : class_weights[i] for i in range(len(class_weights))}
print(class_weights)

"""Запуск обучения НС"""
histoty = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator.filenames) // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator.filenames) // batch_size,
    class_weight=class_weights,
    callbacks=[
          #Сохраняем лучшую эпоху обучения
          ModelCheckpoint(cpt_dir + '/{epoch:02d}_acc_{val_categorical_accuracy:.2f}.h5', monitor='val_categorical_accuracy', verbose=True, save_best_only=True), 
    ]
)

Train generator: 3311 pics
Val generator: 150 pics
Epochs: 50
Batch_size: 18

{0: 3.8057471264367817, 1: 1.7005649717514124, 2: 0.4652894884766723}
Epoch 1/50
183/183 [==============================] - 99s 529ms/step - loss: 1.3115 - categorical_accuracy: 0.3808 - val_loss: 1.3369 - val_categorical_accuracy: 0.3472

Epoch 00001: val_categorical_accuracy improved from -inf to 0.34722, saving model to chkpt/N_H_L_MobileNet/01_acc_0.35.h5
Epoch 2/50
111/183 [=================>............] - ETA: 37s - loss: 1.2141 - categorical_accuracy: 0.3886

KeyboardInterrupt: 

In [9]:
#!g1.1


In [ ]:
#!g1.1
